In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import sys
import time
import pickle
import pandas as pd
import numpy as np
import pylab
import statsmodels.api as sm
#from statsmodels import api as smf
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
#import seaborn as sns
import seaborn.apionly as sns
%matplotlib inline
#matplotlib.rcdefaults()
#matplotlib.verbose.set_level('silent')

In [ ]:
datapath = "../input/"

In [ ]:
phone_brand = pd.read_csv(os.path.join(datapath,'phone_brand_device_model.csv'))
df_test = pd.read_csv(os.path.join(datapath,'gender_age_test.csv'))
df_train = pd.read_csv(os.path.join(datapath,'gender_age_train.csv'))
events = pd.read_csv(os.path.join(datapath,'events.csv'))
app_events = pd.read_csv(os.path.join(datapath,'app_events.csv'))
app_labels = pd.read_csv(os.path.join(datapath,'app_labels.csv'))
label_categories = pd.read_csv(os.path.join(datapath,'label_categories.csv'))

In [ ]:
agp2 = pd.merge(phone_brand, df_train, on='device_id')
agp2_t = pd.merge(phone_brand, df_test, on='device_id')

In [ ]:
mapping2 = (phone_brand.phone_brand.unique())

In [ ]:
mapping_hash = {}

In [ ]:
for index,x in enumerate(mapping2):
    mapping_hash[x] = index

In [ ]:
agp2['phone_brand'] = agp2['phone_brand'].map(mapping_hash).astype(int)
agp2_t['phone_brand'] = agp2_t['phone_brand'].map(mapping_hash).astype(int)

In [ ]:
mapping_hash2={}
mapping2_models =  (phone_brand.device_model.unique())
for index,x in enumerate(mapping2_models):
    mapping_hash2[x] = index

In [ ]:
agp2['device_model'] = agp2['device_model'].map(mapping_hash2).astype(int)
agp2_t['device_model'] = agp2_t['device_model'].map(mapping_hash2).astype(int)

In [ ]:
lista=agp2.phone_brand.unique()

In [ ]:
hsh = {}
def map_f(x):
    if x<22:
        return 0
    if x==23:
        return 1
    if x>=24 and x<=26:
        return 2
    if x==27 or x==28:
        return 3
    if x>28 and x<=31:
        return 4
    if x==32:
        return 5
    if x>32 and x<=38:
        return 6
    if x>38 and x<=42:
        return 7
    if x>42:
        return 8
for index,value in enumerate(lista):
    hsh[value]=map_f(value)

agp2["age"] = agp2["age"].map(hsh).astype(int)

In [ ]:
agp2['gender'] = agp2['gender'].map({'M':0,'F':1}).astype(int)

In [ ]:
agp2.age

In [ ]:
train_data=agp2.values
test_data = agp2_t.values
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[0::,[1,2]])
y_train = train_data[0::,[3,4]]
X_test = scaler.fit_transform(test_data[0::,1::])

In [ ]:
train_data[0::,[1,2]][0]


In [ ]:
y_train[0]

In [ ]:
from sklearn import cross_validation
X_traincv, X_testcv, y_traincv, y_testcv = cross_validation.train_test_split(X_train, 
                                                                             y_train, 
                                                                             test_size=0.4, 
                                                                             random_state=0)

In [ ]:
cv = cross_validation.KFold(len(X_train), n_folds=10)
#cv = cross_validation.ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)

def mean_scores_cv(clf, cv, X, y):
    scores = cross_validation.cross_val_score(clf, X, y, 
                                              scoring=None, 
                                              cv=cv, 
                                              n_jobs=1,
                                              verbose=0,
                                              fit_params=None,
                                              pre_dispatch='2*n_jobs')
    return scores.mean()

#http://scikit-learn.org/stable/modules/ensemble.html
#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
#http://blog.yhathq.com/posts/random-forests-in-python.html
#http://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators=400, 
                              criterion='gini', 
                              max_depth=None, 
                              min_samples_split=2, 
                              min_samples_leaf=1, 
                              min_weight_fraction_leaf=0.0, 
                              max_features='auto', 
                              max_leaf_nodes=None, 
                              bootstrap=True, 
                              oob_score=False, 
                              n_jobs=-1, 
                              random_state=0, 
                              verbose=0, 
                              warm_start=False, 
                              class_weight=None).fit(X_traincv, y_traincv)



eval2_tts = clf2.score(X_testcv, y_testcv)
print(eval2_tts)

eval2_cv = mean_scores_cv(clf2, cv, X_train, y_train)
print(eval2_cv)

# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier
clf3 = DecisionTreeClassifier(criterion='gini', 
                              splitter='best', 
                              max_depth=None, 
                              min_samples_split=1, 
                              min_samples_leaf=1, 
                              min_weight_fraction_leaf=0.0, 
                              max_features=None, 
                              random_state=0, 
                              max_leaf_nodes=None, 
                              class_weight=None, 
                              presort=False).fit(X_traincv, y_traincv)

eval3_tts = clf3.score(X_testcv, y_testcv)
print(eval3_tts)

eval3_cv = mean_scores_cv(clf3, cv, X_train, y_train)
print(eval3_cv)

#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
from sklearn.ensemble import ExtraTreesClassifier
clf4 = ExtraTreesClassifier(n_estimators=300,
                            max_depth=None,
                            min_samples_split=1,
                            random_state=0).fit(X_traincv, y_traincv)

eval4_tts = clf4.score(X_testcv, y_testcv)
print(eval4_tts)

eval4_cv = mean_scores_cv(clf4, cv, X_train, y_train)
print(eval4_cv)

.apply(X_test)**strong text**

In [ ]:
X_test_1 = X_test[0:56203,0::]
X_test_2 = X_test[56204:112406,0::]
X_test_1.shape

In [ ]:
a = clf2.predict(X_test_1)
b = clf2.predict(X_test_2)

In [ ]:
import csv
output = datapath+"output.csv"
outputFile = open('output.csv', 'w', newline='')


In [ ]:
outputWriter = csv.writer(outputFile)


In [ ]:
for x in a:
    outputWriter.writerow(x)

In [ ]:
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
 outputFile.close()